<a href="https://colab.research.google.com/github/bos-hieu/ai-playground/blob/main/product-recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain sentence-transformers faiss-cpu openai tiktoken

In [ ]:
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os

# Sample furniture database (replace with your catalog)
furniture = [
    {"name": "ErgoDesk Pro", "type": "standing desk", "price": 450, "material": "bamboo", "description": "Adjustable height desk with bamboo top."},
    {"name": "ComfortChair X", "type": "ergonomic chair", "price": 350, "material": "mesh", "description": "Breathable mesh chair with lumbar support."},
    {"name": "ZenPod Mini", "type": "office pod", "price": 1200, "material": "acoustic foam", "description": "Soundproof pod for focused work."},
    {"name": "EcoDesk Lite", "type": "standing desk", "price": 300, "material": "MDF", "description": "Budget-friendly standing desk with electric height adjustment."},
    {"name": "AirLift Desk", "type": "standing desk", "price": 600, "material": "steel", "description": "Heavy-duty desk with dual motors."},
    {"name": "CloudChair Elite", "type": "ergonomic chair", "price": 500, "material": "leather", "description": "Premium leather chair with memory foam."},
    {"name": "FocusPod Pro", "type": "office pod", "price": 1500, "material": "glass", "description": "Modern glass pod with noise cancellation."}
]

# 1. Preprocess data into LangChain Documents
docs = [
    Document(
        page_content=f"Name: {f['name']}\nType: {f['type']}\nPrice: ${f['price']}\nMaterial: {f['material']}\nDescription: {f['description']}",
        metadata={"name": f["name"], "price": f["price"], "type": f["type"]}
    ) for f in furniture
]

# 2. Create embeddings and vector store
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(docs, embedder)

# 3. Initialize LLM (replace with your OpenAI key or local model)
os.environ["OPENAI_API_KEY"] = "your-api-key"  # Replace with your OpenAI API key
llm = OpenAI(temperature=0.3)  # Lower temp = more focused responses

# 4. Create recommendation chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_db.as_retriever(search_kwargs={"k": 3}),  # Retrieve top 3 results
    chain_type="stuff"
)

# 5. Test a query
query = "Find a standing desk under $500 with a bamboo top"
result = qa_chain.run(query)
print(f"Recommendation: {result}")